In [1]:
import numpy as np
import pandas as pd
import graph_tool.all as gt
import matplotlib.pyplot as plt

In [2]:
mak = pd.read_csv('map_article_keyword.csv', delimiter=';')
mak.head()

,keyword_id,article_id
0,0,24047
1,0,85073
2,0,86386
3,0,86916
4,0,86919


In [3]:
mak = pd.merge(mak.groupby('keyword_id').count().query('article_id > 10 and article_id < 100'),
         mak, left_index=True, right_on='keyword_id', suffixes=('x', ''))[['keyword_id', 'article_id']]
mak.head()

,keyword_id,article_id
0,0,24047
1,0,85073
2,0,86386
3,0,86916
4,0,86919


In [4]:
kpair = (pd.merge(mak, mak, on='article_id', copy=False)
    .query('keyword_id_x != keyword_id_y')[['keyword_id_x', 'keyword_id_y']])
kpair.head()

,keyword_id_x,keyword_id_y
1,0,6949
2,0,55687
3,0,67861
4,0,76497
5,6949,0


In [5]:
seen = set()
kpair = np.array([((x, y), seen.add((x, y)))[0]
               for (x, y) in kpair.values if (y, x) not in seen])
kpair

array([[     0,   6949],
       [     0,  55687],
       [     0,  67861],
       ..., 
       [100061, 100062],
       [100449, 100504],
       [100530, 100585]])

In [6]:
kpair.shape

(726830, 2)

In [7]:
kname = pd.read_csv('keyword.csv', delimiter=';').query('keyword_id != 0 and keyword_id != 1')
kname.head()

,keyword_id,keyword_name
2,2,aab
3,3,aac
4,4,aadc
5,5,aaffarsuaq
6,6,aafrotropical


In [8]:
kname = pd.merge(pd.DataFrame(mak['keyword_id'].unique()), kname, left_on=0,
                 right_on='keyword_id')[['keyword_id', 'keyword_name']]
kname.head()

,keyword_id,keyword_name
0,15,aalge
1,31,aas
2,39,aba
3,46,abandoned
4,47,abandonment


In [9]:
kpairname = pd.merge(pd.merge(pd.DataFrame(kpair), kname, left_on=0, right_on='keyword_id'),
            kname, left_on=1, right_on='keyword_id')[['keyword_name_x', 'keyword_name_y']]

In [10]:
np.save('kpair', kpair)
np.save('kname', kname)
np.save('kpairname', kpairname)